In [ ]:

import pandas as pd
import numpy as np

In [ ]:
#Reading the data --> Enter the path of the desired data in quotes
data = pd.read_csv('dummy.csv')

In [ ]:
data

In [36]:
data.fillna('None',inplace=True)

We need to put categories instead of the categories of each column

In [38]:
data.head()

,CIF_NO,CODE_GENDER,NAME_EDUCATION_TYPE,FLAG_OWN_REALTY,NAME_HOUSING_TYPE,ORGANIZATION_TYPE,CNT_CHILDREN,OCCUPATION_TYPE,AMT_INCOME_TOTAL,DAYS_EMPLOYED,CREDIT_LIMIT
0,73122693,M,Secondary / secondary special,Y,House / apartment,Business Entity Type 3,0,Laborers,202500.0,-637,0
1,73152323,F,Higher education,N,House / apartment,School,0,Core staff,270000.0,-1188,20000000
2,73171319,M,Secondary / secondary special,Y,House / apartment,Government,0,Laborers,67500.0,-225,4000000
3,73266627,F,Secondary / secondary special,Y,House / apartment,Business Entity Type 3,0,Laborers,135000.0,-3039,60000000
4,73266639,M,Secondary / secondary special,Y,House / apartment,Religion,0,Core staff,121500.0,-3038,5000000


As you can see, there is a credit limit assigned here which makes this data applicable to regression problems. 

In [8]:
#One-hot encoding the data

In [39]:
data_encoded = pd.get_dummies(data)

In [10]:
data_encoded.columns

Index(['CIF_NO', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_EMPLOYED',
       'CREDIT_LIMIT', 'CODE_GENDER_F', 'CODE_GENDER_M',
       'NAME_EDUCATION_TYPE_Academic degree',
       'NAME_EDUCATION_TYPE_Higher education',
       'NAME_EDUCATION_TYPE_Incomplete higher',
       'NAME_EDUCATION_TYPE_Lower secondary',
       'NAME_EDUCATION_TYPE_Secondary / secondary special',
       'FLAG_OWN_REALTY_N', 'FLAG_OWN_REALTY_Y',
       'NAME_HOUSING_TYPE_Co-op apartment',
       'NAME_HOUSING_TYPE_House / apartment',
       'NAME_HOUSING_TYPE_Municipal apartment',
       'NAME_HOUSING_TYPE_Office apartment',
       'NAME_HOUSING_TYPE_Rented apartment', 'NAME_HOUSING_TYPE_With parents',
       'ORGANIZATION_TYPE_Advertising', 'ORGANIZATION_TYPE_Agriculture',
       'ORGANIZATION_TYPE_Bank', 'ORGANIZATION_TYPE_Business Entity Type 1',
       'ORGANIZATION_TYPE_Business Entity Type 2',
       'ORGANIZATION_TYPE_Business Entity Type 3',
       'ORGANIZATION_TYPE_Cleaning', 'ORGANIZATION_TYPE_Constru

In [ ]:
from sklearn.model_selection import train_test_split

For this to work, we first need to take X and y properly where:

    X -> features (in this case, data.loc[:,'CIF_NO':'DAYS_EMPLOYED']
    y -> target variable (in this case , data[:,'CREDIT_LIMIT'])

In [12]:
X = data_encoded.loc[:, data_encoded.columns != 'CREDIT_LIMIT']

In [20]:
y = data_encoded.loc[:,'CREDIT_LIMIT']

In [41]:
from tpot import TPOTRegressor

In [42]:
y

0              0
1       20000000
2        4000000
3       60000000
4        5000000
          ...   
8558     2000000
8559           0
8560     4000000
8561           0
8562    20000000
Name: CREDIT_LIMIT, Length: 8563, dtype: int64

In [44]:
X

,CIF_NO,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_EMPLOYED,CODE_GENDER_F,CODE_GENDER_M,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_None,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff
0,73122693,0,202500.0,-637,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,73152323,0,270000.0,-1188,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,73171319,0,67500.0,-225,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,73266627,0,135000.0,-3039,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,73266639,0,121500.0,-3038,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8558,74932789,1,63000.0,-2791,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8559,74888041,0,225000.0,-7059,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8560,73546253,0,247500.0,-1364,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8561,74252323,1,63000.0,-2791,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [45]:
#Splitting into train and test : Here you can define your own random state and test size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [47]:
tpot = TPOTRegressor(generations=8, population_size=20,
                         offspring_size=None, mutation_rate=0.9,
                         crossover_rate=0.1,
                         scoring='neg_mean_squared_error', cv=5,
                         subsample=0.5, n_jobs=-1,
                         max_time_mins=None, max_eval_time_mins=1.5,
                         random_state=None, config_dict=None,
                         template=None,
                         warm_start=False,
                         memory=None,
                         use_dask=False,
                         periodic_checkpoint_folder=None,
                         early_stop=None,
                         verbosity=3,
                         disable_update_check=False)
tpot.fit(X_train, y_train)
print("Accuracy is {}%".format(tpot.score(X_test, y_test)*100))

30 operators have been imported by TPOT.


_pre_test decorator: _mate_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False.
Generation 1 - Current Pareto front scores:
-1	-662337641430358.5	ElasticNetCV(input_matrix, ElasticNetCV__l1_ratio=0.5, ElasticNetCV__tol=0.1)

_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False.
_pre_test decorator: _random_mutation_operator: num_test=0 l2 was provided as affinity. Ward can only work with euclidean distances..
_pre_test decorator: _random_mutation_operator: num_test=0 '(slice(None, None, None), 

training data did not have the following fields: f86, f62, f29, f42, f11, f82, f37, f52, f64, f93, f9, f7, f39, f55, f46, f83, f66, f3, f56, f2, f65, f80, f44, f95, f47, f14, f12, f16, f6, f76, f69, f54, f58, f51, f18, f63, f45, f79, f49, f17, f27, f33, f5, f90, f67, f13, f92, f41, f36, f57, f53, f50, f75, f38, f87, f15, f72, f81, f20, f84, f43, f21, f91, f8, f78, f4, f0, f77, f28, f19, f22, f30, f1, f68, f24, f70, f34, f35, f71, f25, f26, f10, f89, f32, f31, f85, f94, f23, f60, f88, f59, f48, f73, f61, f40, f74.
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False.
_pre_test decorator: _random_mutation_operator: num_test=1 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments:

training data did not have the following fields: f86, f62, f29, f42, f11, f82, f37, f52, f64, f93, f9, f7, f39, f55, f46, f83, f66, f3, f56, f2, f65, f80, f44, f95, f47, f14, f12, f16, f6, f76, f69, f54, f58, f51, f18, f63, f45, f79, f49, f17, f27, f33, f5, f90, f67, f13, f92, f41, f36, f57, f53, f50, f75, f38, f87, f15, f72, f81, f20, f84, f43, f21, f91, f8, f78, f4, f0, f77, f28, f19, f22, f30, f1, f68, f24, f70, f34, f35, f71, f25, f26, f10, f89, f32, f31, f85, f94, f23, f60, f88, f59, f48, f73, f61, f40, f74.
_pre_test decorator: _random_mutation_operator: num_test=0 l1 was provided as affinity. Ward can only work with euclidean distances..
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Generation 5 - Current Pareto front scores:
-1	-662337641430358.5	ElasticNetC

training data did not have the following fields: f86, f62, f29, f42, f11, f82, f37, f52, f64, f93, f9, f7, f39, f55, f46, f83, f66, f3, f56, f2, f65, f80, f44, f95, f47, f14, f12, f16, f6, f76, f69, f54, f58, f51, f18, f63, f45, f79, f49, f17, f27, f33, f5, f90, f67, f13, f92, f41, f36, f57, f53, f50, f75, f38, f87, f15, f72, f81, f20, f84, f43, f21, f91, f8, f78, f4, f0, f77, f28, f19, f22, f30, f1, f68, f24, f70, f34, f35, f71, f25, f26, f10, f89, f32, f31, f85, f94, f23, f60, f88, f59, f48, f73, f61, f40, f74.
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Automatic alpha grid generation is not supported for l1_ratio=0. Please supply a grid by providing your estimator with the appropriate `alphas=` argument..
_pre_test decorator: _random_mutation_operator: num_test=0 l2 was provided as affinity. Ward can only work with euclidean dist

In [49]:
print("Accuracy is {}%".format(tpot.score(X_test, y_test)*100)) 


Accuracy is -6.47243603915791e+16%
